In [31]:
import torch
import numpy as np
import cv2
import os
from time import time
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision import datasets, transforms, models


In [6]:
labels_dict = {1: 'Eurasian_jay',
 2: 'great_spotted_woodpecker',
 3: 'greenfinch',
 4: 'blue_tit',
 5: 'Carduelis',
 6: 'common_redpoll',
 7: 'great_tit',
 8: 'bullfinch',
 9: 'Eurasian_siskin',
 10: 'Eurasian_tree_sparrow',
 11: 'hawfinch',
 12: 'willow_tit',
 13: 'Fieldfare',
 14: 'Common chaffinch'}

In [ ]:
'''
#load Faster R-CNN without weights
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False)
#load weights
model.load_state_dict(torch.load('/home/costia/birds/model_g2400_6_ep.pt'))

model = model.to(device)
'''

In [ ]:
#TODO Russian, English an japanese version of birds Name

In [43]:
class BirdsDetection:
    """
    Class implements detection with Faster R-CNN trained for 14 birds species
    """

    def __init__(self, source, out_file="detection_Video.avi",
                       weights_path = '/home/costia/birds/model_g2400_6_ep.pt',
                       conf_lvl = 0.6,labels_dict = labels_dict):
        """
        
        """
        self.source = source
        self.weights_path = weights_path
        self.model = self.load_model()
        self.out_file = out_file
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.conf_lvl = conf_lvl
        self.transform = transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                        ])


    def get_video_from_source(self):
        """
        Creates a new video streaming object to extract video frame by frame to make prediction on.
        :returns: opencv2 video capture object
        """
        return cv2.VideoCapture(self.source)

    def load_model(self):
        """
        load Faster R-CNN without weights pytorch hub and  
        load trained weights.
               
        :returns: Pytorch model with weights.
        """
        #
        model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False)
        in_features = model.roi_heads.box_predictor.cls_score.in_features
        num_classes = 14+1 #(n_classes + background)
        model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
       #load weights
        
        model.load_state_dict(torch.load(self.weights_path))
        model.eval()
        return model

    def score_frame(self, frame):
        """
        Takes a single frame as input, and scores the frame .
        :param frame: input frame in numpy/list/tuple format.
        :returns: Labels and  boxes of objects detected by model in the frame.
        """
        self.model.to(self.device)
        
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        inputs = self.transform(frame)
        
        inputs = (inputs) #for img in frame]
        #plt.imshow(inputs)
        inputs.unsqueeze_(0)
        #print(inputs.shape)
        inputs = inputs.to(self.device) #for img in inputs]
        with torch.no_grad():
            preds = self.model(inputs)
        torch.cuda.empty_cache()
        
        return preds

    def class_to_label(self, label):
        """
        For a given label value, return bird species name.
        :param x: numeric label
        :return: corresponding bird species
        """
        return labels_dict [label]

    def plot_boxes(self, preds, frame):
        """
        Takes a frame and its results as input, and plots the bounding boxes and label on to the frame.
        :param results: contains labels and coordinates predicted by model on the given frame.
        :param frame: Frame which has been scored.
        :returns: Frame with bounding boxes and labels ploted on it.
        """
        
        
        mask = preds[0]['scores'] >= self.conf_lvl
        print('Founded:',preds[0]['scores'])
        confidences = preds[0]['scores'][mask]
        boxes = preds[0]['boxes'][mask]
        labels = preds[0]['labels'][mask]
        
        flag = True
        
        for i, box in enumerate(boxes):
            x_min, y_min, x_max, y_max = map(int,box)
            bgr = (0, 255, 0)
            #print((x_min, y_min), (x_max, y_max))
            cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), bgr, 2)
            cv2.putText(frame, self.class_to_label(int(labels[i])), (x_min, y_min), cv2.FONT_HERSHEY_SIMPLEX, 0.9, bgr, 2)
        return frame
      

    def __call__(self):
        """
        This function is called when class is executed, it runs the loop to read the video frame by frame,
        and write the output into a new file.
        :returns: void
        """
        player = self.get_video_from_source()
        assert player.isOpened()
        x_shape = int(player.get(cv2.CAP_PROP_FRAME_WIDTH))
        y_shape = int(player.get(cv2.CAP_PROP_FRAME_HEIGHT))
        four_cc = cv2.VideoWriter_fourcc(*"MJPG")
        out = cv2.VideoWriter(self.out_file, four_cc, 30, (x_shape, y_shape))
        i = 0
        while True:
            
            start_time = time()
            try:
                ret, frame = player.read()
            except AssertionError:
                print("Video ended")
                break
            assert ret
            if i%1==0:
                results = self.score_frame(frame)
                frame = self.plot_boxes(results, frame)
                end_time = time()
                fps = 1/np.round(end_time - start_time, 3)
                print(f"Frames Per Second : {fps}")
            
            i+=1
            out.write(frame)



In [ ]:
# Create a new birds detection object and execute.
detect = BirdsDetection("/home/costia/VID-20211031-WA0001.mp4",conf_lvl=0.2,
                        weights_path = '/home/costia/birds/best_model.pt',
                        out_file='detected_birds__test.avi')
detect()

Founded: tensor([0.1180, 0.1160, 0.1086], device='cuda:0')
Frames Per Second : 3.278688524590164
Founded: tensor([0.1180, 0.1160, 0.1086], device='cuda:0')
Frames Per Second : 5.154639175257731
Founded: tensor([0.1180, 0.1160, 0.1086], device='cuda:0')
Frames Per Second : 5.0761421319796955
Founded: tensor([0.1180, 0.1160, 0.1086], device='cuda:0')
Frames Per Second : 5.05050505050505
Founded: tensor([0.1180, 0.1160, 0.1086], device='cuda:0')
Frames Per Second : 5.0
Founded: tensor([0.1095, 0.1077, 0.0959], device='cuda:0')
Frames Per Second : 4.9504950495049505
Founded: tensor([0.1103, 0.1063, 0.1028], device='cuda:0')
Frames Per Second : 4.366812227074235
Founded: tensor([0.1114, 0.1083, 0.0997], device='cuda:0')
Frames Per Second : 5.291005291005291
Founded: tensor([0.1085, 0.1003, 0.0910], device='cuda:0')
Frames Per Second : 2.5510204081632653
Founded: tensor([0.0997, 0.0940, 0.0814], device='cuda:0')
Frames Per Second : 3.558718861209964
Founded: tensor([0.1069, 0.0743, 0.0582], 